In [1]:
import numpy as np
from keras.datasets import imdb

In [30]:
top_words = 5000
(xtrain, ytrain), (xtest, ytest) = imdb.load_data(num_words=top_words)

In [31]:
print(xtrain.shape, xtest.shape)
print(ytrain.shape, ytest.shape)

(25000,) (25000,)
(25000,) (25000,)


In [32]:
print(xtrain[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [33]:
# imdb.get_word_index()

In [34]:
dict_w2n = imdb.get_word_index()

dict_n2w = { n+3:w for (w,n) in dict_w2n.items() }

In [35]:
r1 = xtrain[0]
review = [ dict_n2w.get(i) for i in r1]
print(review)

[None, 'this', 'film', 'was', 'just', 'brilliant', 'casting', 'location', 'scenery', 'story', 'direction', "everyone's", 'really', 'suited', 'the', 'part', 'they', 'played', 'and', 'you', 'could', 'just', 'imagine', 'being', 'there', 'robert', None, 'is', 'an', 'amazing', 'actor', 'and', 'now', 'the', 'same', 'being', 'director', None, 'father', 'came', 'from', 'the', 'same', 'scottish', 'island', 'as', 'myself', 'so', 'i', 'loved', 'the', 'fact', 'there', 'was', 'a', 'real', 'connection', 'with', 'this', 'film', 'the', 'witty', 'remarks', 'throughout', 'the', 'film', 'were', 'great', 'it', 'was', 'just', 'brilliant', 'so', 'much', 'that', 'i', 'bought', 'the', 'film', 'as', 'soon', 'as', 'it', 'was', 'released', 'for', None, 'and', 'would', 'recommend', 'it', 'to', 'everyone', 'to', 'watch', 'and', 'the', 'fly', None, 'was', 'amazing', 'really', 'cried', 'at', 'the', 'end', 'it', 'was', 'so', 'sad', 'and', 'you', 'know', 'what', 'they', 'say', 'if', 'you', 'cry', 'at', 'a', 'film', 'i

In [36]:
from keras.preprocessing import sequence

max_review_length = 500

Xtrain = sequence.pad_sequences(xtrain, maxlen=max_review_length)
Xtest = sequence.pad_sequences(xtest, maxlen=max_review_length)

print(Xtrain.shape)
print(Xtest.shape)

(25000, 500)
(25000, 500)


In [37]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding

In [38]:
embedding_vector_length = 32

In [11]:
model = Sequential()

model.add(Embedding(top_words,
                   embedding_vector_length,
                   input_length = max_review_length))

# model.add(Conv1D(filters=32,
#                 kernel_size=3,
#                 padding='same',
#                 activation='relu'))

# model.add(MaxPool1D(pool_size=2))

model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [13]:
model.fit(Xtrain, ytrain,
         epochs=3, 
         batch_size=64,)

Epoch 1/3
391/391 [==============================] - 444s 1s/step - loss: 0.4854 - accuracy: 0.7576
Epoch 2/3
391/391 [==============================] - 473s 1s/step - loss: 0.3119 - accuracy: 0.8731
Epoch 3/3
391/391 [==============================] - 501s 1s/step - loss: 0.2585 - accuracy: 0.8982


In [39]:
def predict(review):
    rev = review.split()
    rev1 = [ dict_w2n.get(w)+3 for w in rev ]
    rev2 = [0] * (500 - len(rev1) - 1) + [1] + rev1
    rev3 = np.array(rev2).reshape(1, -1)
    return model.predict(rev3)

In [43]:
revPos = 'the movie was absolutely good just brilliant the cast was great'
revNeg = 'the movie was the worst the story was so bad the movie was so boring'

In [44]:
print(predict(revPos))

[[0.8940871]]


In [45]:
review = predict('i am happy')

In [47]:
print(review)

[[0.5130987]]


In [48]:
print(predict('i am sad'))

[[0.1994696]]


In [50]:
print(predict('i am sad'))

[[0.1994696]]
